In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
pd.options.display.max_columns = None
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, precision_score, recall_score,  accuracy_score, precision_recall_curve
import seaborn as sns
import nltk
import re
import tensorflow as tf
import keras 
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers import SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.metrics import confusion_matrix


2024-04-10 10:39:49.544134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 10:39:49.582188: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 10:39:49.582792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 10:39:50.228755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("labeled_data.csv",delimiter=',',engine='python',encoding='utf-8-sig')


In [3]:
data.loc[data["class"] == 0, "class"] = 1 
data.loc[data["class"]== 2, "class"] = 0

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('all')
import string
stemmer = nltk.SnowballStemmer('english')
stopword = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nlplab1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/nlplab1/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/nlplab1/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/nlplab1/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/nlplab1/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /home/n

In [5]:
def data_cleaning(words):
    words = str(words).lower()
    words = re.sub('@[\w]+','',words)
    words = re.sub('\[.*?/]', "", words)
    words = re.sub('https?://\S+|www\.\S+', "", words)
    words = re.sub('<.*?=+->^$#@!%', "", words)
    words = re.sub('\n', "", words)
    words = [word for word in words if word not in string.punctuation]
    words = "".join(words)
    words = [word for word in words.split(' ') if word not in stopword]
    words = " ".join(words)
    words = [stemmer.stem(word) for word in words.split(' ')]
    words = " ".join(words)
    
    return words

In [6]:
data['tweet'] = data['tweet'].apply(data_cleaning)
data['tweet'].head(10)

0     rt  woman shouldnt complain clean hous amp ma...
1     rt  boy dat coldtyga dwn bad cuffin dat hoe 1...
2     rt  dawg rt  ever fuck bitch start cri confus...
3                                rt   look like tranni
4     rt  shit hear might true might faker bitch to...
5     shit blow meclaim faith somebodi still fuck h...
6               sit hate anoth bitch  got much shit go
7    8220 caus im tire big bitch come us skinni gir...
8                amp might get ya bitch back amp that 
9                       hobbi includ fight mariambitch
Name: tweet, dtype: object

In [7]:
x = data['tweet']
y = data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 50000
max_len = 300

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(x_train)

sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = pad_sequences(sequences, maxlen = max_len)

In [9]:
from tensorflow.keras.models import Sequential
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers import SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.metrics import confusion_matrix


In [10]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_shape=(300,)))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,080,501
Trainable params: 5,080,501
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(sequences_matrix, y_train, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
124/124 [==============================] - 37s 286ms/step - loss: 0.3554 - accuracy: 0.8569 - val_loss: 0.1906 - val_accuracy: 0.9344
Epoch 2/5
124/124 [==============================] - 38s 304ms/step - loss: 0.1353 - accuracy: 0.9485 - val_loss: 0.1333 - val_accuracy: 0.9468
Epoch 3/5
124/124 [==============================] - 34s 273ms/step - loss: 0.1092 - accuracy: 0.9616 - val_loss: 0.1306 - val_accuracy: 0.9476
Epoch 4/5
124/124 [==============================] - 36s 294ms/step - loss: 0.0818 - accuracy: 0.9719 - val_loss: 0.1380 - val_accuracy: 0.9460
Epoch 5/5
124/124 [==============================] - 37s 302ms/step - loss: 0.0705 - accuracy: 0.9762 - val_loss: 0.1367 - val_accuracy: 0.9455


In [12]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len )

In [13]:
acc = model.evaluate(test_sequences_matrix, y_test)

155/155 [==============================] - 3s 22ms/step - loss: 0.1359 - accuracy: 0.9461


In [14]:
lstm_prediicton = model.predict(test_sequences_matrix)

155/155 [==============================] - 3s 21ms/step


In [15]:
res = []
for prediction in lstm_prediicton:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [16]:
print(confusion_matrix(y_test,res))

[[ 729  127]
 [ 140 3961]]


In [17]:
model.save("model.h5")

In [18]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
import tensorflow.keras as keras
load_model=keras.models.load_model("model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [20]:
test = 'you are an idiot'

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

test=[clean_text(test)]
print(test)

seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
print(seq)

pred = load_model.predict(padded)

print("pred", pred)
if pred<0.5:
    print("not offensive")
else:
    print("offensive")
    

you are an idiot
you are an idiot
['idiot']
[[1293]]
1/1 [==============================] - 0s 155ms/step
pred [[0.6279899]]
offensive


EXPLANATION